In [5]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import requests
from bs4 import BeautifulSoup 
import re 
import time
import dateparser
#Le JDD : Plutôt à droite

class JDD:

    def __init__(self):
        self.articals = {}
        self.dates = []
        
    def download_arc_names(self, categorie = 'all', quantity = 50):
        n = int(round(quantity/50+0.5))
        #выгружаем страницы
        categories = 'Politique, Societe, International, Economie, JDD-Paris, Culture, Sport, Sondages'.split(', ')

        if categorie=='all':
            pages=[]
            for cat in categories:
                page_ = [requests.get("https://www.lejdd.fr/"+cat+'/'+str(page_number)) for page_number in range(1,n)]
                pages.extend(page_)
                print(pages)
        else:
            pages = [requests.get("https://www.lejdd.fr/"+categorie+'/'+str(page_number)) for page_number in range(1,n)]
            
        # список названий статей и ссылок
        titles=[]
        for page in pages:
            one_page_ts=BeautifulSoup(page.text, "html5lib").find_all('h3')
            titles.extend(one_page_ts)

        links=["https://www.lejdd.fr"+l("a")[0]["href"] for l in titles]
        n_l=dict(zip([i.text for i in titles], links))
        return n_l
    
    def get_art(self, categorie = 'all', quantity = 50):
        # время печати и тексты статей 
        texts = []
        n_l = self.download_arc_names(categorie, quantity)
        for name, i in list(n_l.items()):
            resp = requests.get(i)
            #time.sleep(0.3)
            bs = BeautifulSoup(resp.text, 'html5lib')

            t = BeautifulSoup(resp.text, "html5lib").find_all('time')[0]
            t = ' '.join(t.get_text().split(',')[:2])
            time = dateparser.parse(t)

            text = BeautifulSoup(" ".join([p.text for p in bs.find_all("p")]), "html5lib").get_text().replace('\xa0', ' ')
        
            self.articals[time] = (name, text)
            self.dates.append(time)    
        self.dates.sort()
        print('Corpus of {} articals is done. First article published {}, the last one  - {}'.format(len(self.dates), min(self.dates),max(self.dates)))
        return self.articals
            
    def choose_time_per(self, l_b, r_b):
        l_b = dateparser.parse(l_b)
        r_b = dateparser.parse(r_b)
        for date in self.dates:
            if date >= l_b and date <= r_b :
                self.subcorpus[date] = (self.articals[date][0], self.articals[date][1])
        return self.subcorpus
    
    def clear_subcorpus(self):
        self.subcorpus= []
    
    def write_coprus(self, corpus):
        
        f = open('all_articals.txt', 'w', encoding='utf-8')
        for date,(name, text) in corpus.items():
            f.write(str(date)+'\n')
            f.write(name+'\n')
            f.write(text+'\n')
        f.close()
        print('Corpus({} articls) is written in file all_articals.txt'.format(len(list(corpus.items()))))


In [6]:
jdd = JDD()
jdd.get_art(categorie='Politique', quantity=50)
jdd.write_coprus(jdd.articals)

Corpus of 50 articals is done. First article published 2018-10-08 08:00:00, the last one  - 2018-10-14 17:57:00
Corpus(49 articls) is written in file all_articals.txt
